In [3]:
# Python Review Project

#Applying ML to Cancer Data Analysis

In [1]:
import os

os.chdir('C:/Users/ME/Documents/QBIO/sp24_cw/qbio_490_dgmarsha/analysis_data')

In [2]:

import numpy as np
import pandas as pd
import cptac
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

cptac warning: Your version of cptac (1.1.2) is out-of-date. Latest is 1.5.13. Please run 'pip install --upgrade cptac' to update it. (C:\Users\ME\anaconda3\envs\qbio_490_dgmarsha_v2\Lib\threading.py, line 1010)


In [6]:
cptac.download(dataset="Ccrcc")
ccrcc = cptac.Ccrcc()

KeyError: 'Sample_Tumor_Normal'

In [ ]:
protein_data = ccrcc.get_dataframe("proteomics","umich")
rna_data = ccrcc.get_dataframe("transcriptomics","washu")
#bcm source doesn't work, I think the data is no longer hosted.
clinical_data = ccrcc.get_clinical("mssm")

In [ ]:
#We are comparing data from the clinical, protein, and RNA databases,
#so we need to find the overlap between them. Any comparisons made would mean nothing if 
#they were across different patient's data.
overlap_patients = np.intersect1d(protein_data.index,clinical_data.index)
overlap_patients = np.intersect1d(overlap_patients,rna_data.index)

overlap_protein_data = protein_data.loc[[index in overlap_patients for index in protein_data.index],:]
overlap_rna_data = rna_data.loc[[index in overlap_patients for index in rna_data.index],:]
overlap_clinical_data = clinical_data.loc[[index in overlap_patients for index in clinical_data.index],:]
#These are the 110 patients shared across these three dataframes.

In [ ]:
print(list(overlap_protein_data.index)==list(overlap_rna_data.index) and list(overlap_protein_data.index)==list(overlap_clinical_data.index))
#Indices match.
overlap_rna_data.shape

In [ ]:
#The transcriptomic data has many versions for each gene, which makes this difficult.
#I want to make sure I don't find two versions of a gene are the most DE.
kept_col=[False]*len(overlap_rna_data.columns)
# def zeroes_count(array):
#     count = np.where([i==0 for i in array],True, False)
#     return sum(count)
for i in range(len(overlap_rna_data.columns)):
    if (np.sum(np.isnan(overlap_rna_data[overlap_rna_data.columns[i]]))==0 ):
        # and zeroes_count(overlap_rna_data[overlap_rna_data.columns[i]]<)):
        kept_col[i]=True
overlap_rna_data=overlap_rna_data.loc[:,kept_col]
        #No nulls in the column.
        #I am removing nulls by cutting columns b/c I don't have many patients and I want to
        #keep all of them. If there is one patient null across, that will ruin this.

kept_col=[False]*len(overlap_protein_data.columns)
# used_genes =set()
for i in range(len(overlap_protein_data.columns)):
    if (np.sum(np.isnan(overlap_protein_data[overlap_protein_data.columns[i]]))==0):
        kept_col[i]=True
overlap_protein_data=overlap_protein_data.loc[:,kept_col]

In [ ]:
#The zeroes have to removed before log2 scaling.
def adjust_zeroes(array):
    x = np.where([i ==0 for i in array],0.00001,[i for i in array])
    return x
    for i in range(len(array)):
        if(array[i]==0):
            array[i] = 0.00001
for col in overlap_rna_data.columns:
    overlap_rna_data.loc[:,col]= adjust_zeroes(overlap_rna_data.loc[:,col])
    overlap_rna_data.loc[:,col] = np.log2(overlap_rna_data.loc[:,col])
    overlap_rna_data.loc[:,col] =np.where([-16.609640474436812==val for val in 
                                           overlap_rna_data.loc[:,col]],0,[val for val in 
                                                    overlap_rna_data.loc[:,col]])
#I am putting them back in after though, so as to not have the large negative numbers 
#produced skew the DE.

In [ ]:
mostDiffGene = [["gene","gene1","gene1","gene1","gene1"],[0,0,0,0,0]]
for i in range(len(overlap_rna_data.columns)):
    gene_III = overlap_rna_data.iloc[[x =="Stage III" for x in overlap_clinical_data["tumor_stage_pathological"]],i]
    gene_I = overlap_rna_data.iloc[[x =="Stage I" for x in overlap_clinical_data["tumor_stage_pathological"]],i]
    diff = np.abs(np.mean(gene_III)-np.mean(gene_I))
    rank = 4
    while(diff>mostDiffGene[1][rank]):
        if(rank>0 and diff>mostDiffGene[1][rank-1]):
            rank = rank-1;
            continue
        mostDiffGene[1][rank] = diff
        mostDiffGene[0][rank] = overlap_rna_data.columns[i]

In [ ]:
mostDiffGene

In [ ]:
mostDiffProt = [["gene","gene1","gene1","gene1","gene1"],[0,0,0,0,0]]
for i in range(len(overlap_protein_data.columns)):
    prot_III = overlap_protein_data.iloc[[x =="Stage III" for x in overlap_clinical_data["tumor_stage_pathological"]],i]
    prot_I = overlap_protein_data.iloc[[x =="Stage I" for x in overlap_clinical_data["tumor_stage_pathological"]],i]
    diff = np.abs(np.mean(prot_III)-np.mean(prot_I))
    rank = 4
    while(diff>mostDiffProt[1][rank]):
        if(rank>0 and diff>mostDiffProt[1][rank-1]):
            rank = rank-1;
            continue
        mostDiffProt[1][rank] = diff
        mostDiffProt[0][rank] = overlap_protein_data.columns[i]


In [15]:
Features = pd.DataFrame()
#I'm going to omit the version of each gene and protein for simplicity.
for i in range(5):
    DEprot = mostDiffProt[0][i]
    Features["Protein "+DEprot[0]] = overlap_protein_data[DEprot]
    DEgene = mostDiffGene[0][i]
    Features["Expression of "+DEgene[0]] = overlap_rna_data[DEgene]
target = np.array(overlap_clinical_data["tumor_stage_pathological"])
encode_dict = {'Stage I':[1,0,0,0], 'Stage II':[0,1,0,0], 'Stage III':[0,0,1,0], 'Stage IV':[0,0,0,1]}
encode_dict_alt = {'Stage I':0, 'Stage II':1, 'Stage III':2, 'Stage IV':3}

#One hot, though ordinal probably would have worked. This allows the model to not expect a 
#pattern 1 to 2, 2 to 3, etc. Probably not necessary.
y = np.array([encode_dict[stage] for stage in target])
y_alt = np.array([encode_dict[stage] for stage in target])
X = np.array(Features)


NameError: name 'mostDiffProt' is not defined

In [311]:
classifiers = {
    "KNeighbors Classifier":KNeighborsClassifier(), "DecisionTree Classifier":DecisionTreeClassifier(),
    "MLP Classifier":MLPClassifier(solver = 'lbfgs'), "GaussianNB":GaussianNB()}
#That solver is supposed to be better for small datasets, and it stops the convergence warnings.

for classi in classifiers.keys(): 
    classifier = classifiers[classi]
    accuracy = [-1]*10
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,stratify=y)
        if(classi == "GaussianNB"):
            X_train, X_test, y_train, y_test = train_test_split(X, y_alt, train_size=0.7,stratify=y_alt)
        #The Gaussian classifier can't seem to handle the one-hot encoded target.
        #The others all perform better with it, so I am doing it that way for them.
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test) #Mind X,y are np.array, not pd.series.
        accuracy[i] = (sum(y_pred == y_test) / len(y_test))
    print(np.mean(accuracy),"percent average accuracy with "+classi)



0.8037878787878789 percent average accuracy with KNeighbors Classifier
0.7242424242424242 percent average accuracy with DecisionTree Classifier


C:\Users\ME\anaconda3\envs\qbio_490_dgmarsha\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ME\anaconda3\envs\qbio_490_dgmarsha\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ME\anaconda3\envs\qbio_490_dgmarsha\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ME\anaconda3\envs\qbio_490_dgmarsha\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

0.7999999999999999 percent average accuracy with MLP Classifier


ValueError: y should be a 1d array, got an array of shape (77, 4) instead.

In [299]:
#I don't know the accuracy of the Gaussian model, but of the three I was able to do, 
#KNeighbor was the best.
#If you run this code, it should work, allowing you to see if the last model would have
#taken the top spot.

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]